In [4]:
import django_initializer
import petl as etl
import const

In [5]:
from decimal import Decimal


constraints = [dict(name="street_name", field="street", test=str, optional=False)]
residents = (
    etl.fromcsv(const.DATA_DIR / "residents.csv")
    # .convert("no", Decimal)
    .convert("1920", const.null_values)
    .convert("1921", const.null_values)
    .convert("1922", const.null_values)
    .convert("1923", const.null_values)
    .convert("1920 census", const.null_values)
)
residents

street,no,1920,1921,1922,1923,1920 census,losses,notes
Admiral Blvd.,3,None,None,None,R.W. Williams,,,
Admiral Blvd.,800,None,"Pardillo, Victor",None,None,"Padilla, Victor (25) laborer",,
Admiral Blvd.,801,None,Hooper Bros.*,None,None,,,
Admiral Blvd.,807,None,None,"Hoper Bros, whol. Tea",None,,,
Admiral Blvd.,807,Tulsa Coffee Co.*,Tulsa Coffee Co.*,Tulsa Coffee Co.,None,,,


In [3]:
problems = etl.validate(residents, constraints=constraints)
problems.lookall()

+------+-----+-------+-------+-------+
| name | row | field | value | error |
+======+=====+=======+=======+=======+

In [5]:
from addresses.models import Address, Street
from petl import dicts

year_columns = (
    "1920",
    "1921",
    "1922",
    "1923",
    "1920 census",
)

for row in dicts(residents):
    street, street_created = Street.get_or_create_from_name(row["street"])

    building_number = row["no"]
    if building_number == "" or street is None:
        # logger.info("no building number, skipped: %s", row)
        continue

    address, address_created = Address.get_or_create_from_unique(
        street=street, number=building_number
    )

    print(address, address_created)

3.00 None Admiral Boulevard False
800.00 None Admiral Boulevard False
801.00 None Admiral Boulevard False
807.00 None Admiral Boulevard False
807.00 None Admiral Boulevard False
816.00 None Admiral Boulevard False
816.00 None Admiral Boulevard False
817.00 None Admiral Boulevard False
817.00 None Admiral Boulevard False
817.00 None Admiral Boulevard False
817.00 None Admiral Boulevard False
817.00 None Admiral Boulevard False
818.00 None Admiral Boulevard False
818.00 None Admiral Boulevard False
820.00 None Admiral Boulevard False
822.00 None Admiral Boulevard False
822.00 None Admiral Boulevard False
824.00 None Admiral Boulevard False
8245.00 None Admiral Boulevard False
104.00 None Archer Street East False
104.00 None Archer Street East False
105.00 None Archer Street East False
119.00 None Archer Street East False
119.00 None Archer Street East False
121.00 None Archer Street East False
123.00 None Archer Street East False
123.00 None Archer Street East False
124.00 None Archer St

IntegrityError: null value in column "number" of relation "addresses_address" violates not-null constraint
DETAIL:  Failing row contains (6784, null, None, 111).
